In [1]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
pd.options.display.max_columns = 100
import matplotlib
matplotlib.style.use('ggplot')
import numpy as np
pd.options.display.max_rows = 100
from itertools import islice
import urllib2
from bs4 import BeautifulSoup
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from DatumBox import DatumBox
import re
from urlparse import urlparse
import seaborn as sns # To have more than seven colors in pie diagram
import plotly.plotly as py # For Gantt Chart
import plotly.figure_factory as ff # For Gantt Chart
import sklearn

In [3]:
import urllib
from bs4 import BeautifulSoup
import csv
import urllib2
import re
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet
from itertools import chain
from nltk.corpus import stopwords
from DatumBox import DatumBox
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
datum_box = DatumBox("2a13913dda346761765020c1f66e34f8")
TAG_RE = re.compile(r"<[^>]+>")
nltk.download('punkt')
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
cachedStopWords = stopwords.words("english")

def remove_tags(text):
    return TAG_RE.sub('', text)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

def main_text(url):
    html = urllib.urlopen(url).read()
    soup = BeautifulSoup(html,"html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    text = ' '.join([word for word in text.split() if word not in cachedStopWords])
    for char in text:
        if(not((ord(char) >= 97 and ord(char) <= 122) or (ord(char) >= 65 and ord(char) <= 90))):
            text = text.replace (char," ")
    return text

def openfile(f):
    x = []
    y = []
    finial = []; i=2
    csvfile = open(f,'rb')
    reader = csv.DictReader(csvfile)
    text1='';text2='';text1_main='';text2_main=''
    synonyms=None
    for row in reader:
        synonyms=None
        if i==0:
            print str(round(cosine_sim(text1,text2),2))+ "      " +str(round(cosine_sim(text1_main,text2_main),2))
            i=2
            text1='';text1_main=''
            text2='';text2_main=''
        elif i==1:
            text1 = main_text(row['url'])
            text1_main = text1 + ' '
            for j in text1.split(' '):
                synonyms = wordnet.synsets(j)
                synonyms = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
                for k in synonyms:
                    text1_main += k + ' '
            i-=1
        elif i==2:
            text2 = main_text(row['url'])
            text2_main = text2 + ' '
            for j in text2.split(' '):
                synonyms = wordnet.synsets(j)
                synonyms = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
                for k in synonyms:
                    text2_main += k + ' '
            i-=1
    print a
    csvfile.close()

#openfile('/home/chaitanya/Documents/Codes/IR_Vidhya mam/resources/input/10.csv')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def wordnet_similarity(a,b):
    url1 = a
    url2 = b
    text1='';text2='';text1_main='';text2_main=''
    text1 = main_text(url1)
    text2 = main_text(url2)
    word_count = 0;similar = 0
    text1_split = text1.split(' ');text2_split = text2.split(' ')
    text1_split = [x.upper() for x in text1_split if x]
    text2_split = [x.upper() for x in text2_split if x]
    similar = 0.0;word_count = 0.0;flag=0
    
    for j in text1_split:
        flag = 0
        if j not in text2_split:
            synonyms = wordnet.synsets(j)
            synonyms = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
            for k in synonyms:
                if k in text2_split:
                    flag = 1
        else:
            similar += 1
        if flag == 1:
            similar += 1
        word_count += 1

    for j in text2_split:
        flag = 0
        if j not in text1_split:
            synonyms = wordnet.synsets(j)
            synonyms = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
            for k in synonyms:
                if k in text1_split:
                    flag = 1
        else:
            similar += 1
        if flag == 1:
            similar += 1
        word_count += 1

    #text2_main += k + ' '

    return similar/word_count


In [5]:
disp_data= pd.read_csv('internal_cache.csv')
#disp_data.head()
print len(disp_data)

212


In [6]:
#disp_data = disp_data[disp_data.url != *'google'*]
#disp_data[]
#df[df.line_race != 0]
disp_data = disp_data[~disp_data["url"].str.contains("google")]
disp_data = disp_data[~disp_data["url"].str.contains("home")]

In [7]:
print len(disp_data)

134


In [8]:
disp_data = disp_data[20:40]
disp_data

,Unnamed: 0,id,lastVisitTime,title,url,visitCount,timeSpent,Internal_category,session,sim_val
43,168,516,7/12/2017 6:51:00 PM,Dataset-driven research for improving recommen...,http://dl.acm.org/citation.cfm?id=2090122,1,11.0,General,11,1.000000
44,167,517,7/12/2017 6:51:35 PM,A comparative analysis of offline and online e...,http://dl.acm.org/citation.cfm?id=2532511,1,35.0,General,11,0.036585
45,166,519,7/12/2017 6:52:07 PM,NaN,http://delivery.acm.org/10.1145/2470000/246770...,1,32.0,General,11,0.857143
46,165,521,7/12/2017 6:52:14 PM,NaN,http://delivery.acm.org/10.1145/2100000/209012...,1,7.0,General,11,0.800000
47,164,523,7/12/2017 6:52:19 PM,NaN,http://delivery.acm.org/10.1145/2540000/253251...,1,5.0,General,11,0.014778
48,163,525,7/12/2017 6:57:12 PM,A Recommendation System Based on Hierarchical ...,http://ieeexplore.ieee.org/document/7442097/#f...,1,293.0,General,11,0.071130
51,160,509,7/12/2017 7:05:37 PM,Dataset 2 for Scholarly Paper Recommendation,https://www.comp.nus.edu.sg/~sugiyama/Dataset2...,2,2.0,General,10,0.007673
57,154,510,7/12/2017 7:13:48 PM,Dataset1 for Scholarly Paper Recommendation,http://www.comp.nus.edu.sg/~sugiyama/Dataset1....,3,43.0,General,10,0.155747
65,146,541,7/12/2017 7:27:35 PM,NaN,http://ieeexplore.ieee.org/abstract/document/4...,1,42.0,General,10,0.108497
67,144,543,7/12/2017 7:29:37 PM,Babel,http://babel.eigenfactor.org/,1,59.0,General,10,0.061433


<h1 style="color:red">CLUSTERING </h1>

In [9]:
def url_sim(url1,url2):
        return wordnet_similarity(url1,url2)

In [10]:
clusters = []
temp = []
thres = 0.1

for i in range(0,len(disp_data)):
    curr_url = disp_data.iloc[i]['url']
    
    if (len(clusters) == 0):
        temp.append(curr_url)
        clusters.append(temp)
        print "??a??",
        
    if(len(clusters) > 0):
        c_indx = 0
        max_sim = 0
        for c in range(0,len(clusters)):
            count = 0
            min_lim = url_sim(clusters[c][0],curr_url)
            max_lim = url_sim(clusters[c][len(clusters[c])-1],curr_url)
            if (min_lim > thres or max_lim > thres):
                greatest_lim = max(min_lim,max_lim)
                if (greatest_lim > max_sim):
                    max_sim = greatest_lim
                    c_indx = c
        
        if(max_sim > thres):
            clusters[c_indx].append(curr_url)
            print "??c??",
        else:
            tmp = []
            tmp.append(curr_url)
            clusters.append(tmp)
            print "??d??",
    print '---1---',

??a?? ??c?? ---1--- ??c?? ---1--- ??d?? ---1--- ??c?? ---1--- ??c?? ---1--- ??c?? ---1--- ??c?? ---1--- ??c?? ---1--- ??c?? ---1--- ??d?? ---1--- ??c?? ---1--- ??c?? ---1--- ??d?? ---1--- ??d?? ---1--- ??c?? ---1--- ??c?? ---1--- ??d?? ---1--- ??c?? ---1--- ??c?? ---1--- ??c?? ---1---


In [15]:
for i in clusters:
    print i
    print "\n\n"
#print temp

['http://dl.acm.org/citation.cfm?id=2090122', 'http://dl.acm.org/citation.cfm?id=2090122', 'http://dl.acm.org/citation.cfm?id=2532511']



['http://delivery.acm.org/10.1145/2470000/2467701/p153-sugiyama.pdf?ip=14.139.187.130&id=2467701&acc=ACTIVE%20SERVICE&key=045416EF4DDA69D9%2E8DD97CC74180EE26%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35&CFID=784797565&CFTOKEN=41036537&__acm__=1499865974_286433830c9da409d79f293351437853', 'http://delivery.acm.org/10.1145/2100000/2090122/p44-verbert.pdf?ip=14.139.187.130&id=2090122&acc=ACTIVE%20SERVICE&key=045416EF4DDA69D9%2E8DD97CC74180EE26%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35&CFID=784797565&CFTOKEN=41036537&__acm__=1499865981_7e73cdd5987e2010af88169e1b49f19a', 'http://delivery.acm.org/10.1145/2540000/2532511/p7-beel.pdf?ip=14.139.187.130&id=2532511&acc=ACTIVE%20SERVICE&key=045416EF4DDA69D9%2E8DD97CC74180EE26%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35&CFID=784797565&CFTOKEN=41036537&__acm__=1499865986_abd1d94fb085c488e4cc17a2a35791e4']



['http://ieeexplore.ie

In [11]:
for i in clusters:
    print i
    print "\n\n"
#print temp

['http://dl.acm.org/citation.cfm?id=2090122', 'http://dl.acm.org/citation.cfm?id=2090122', 'http://dl.acm.org/citation.cfm?id=2532511', 'http://ieeexplore.ieee.org/document/7442097/#full-text-section', 'https://www.comp.nus.edu.sg/~sugiyama/Dataset2.html', 'http://www.comp.nus.edu.sg/~sugiyama/Dataset1.html', 'http://ieeexplore.ieee.org/abstract/document/4061472/citations', 'http://ieeexplore.ieee.org/abstract/document/4061472/figures', 'http://ieeexplore.ieee.org/abstract/document/4061472/']



['http://delivery.acm.org/10.1145/2470000/2467701/p153-sugiyama.pdf?ip=14.139.187.130&id=2467701&acc=ACTIVE%20SERVICE&key=045416EF4DDA69D9%2E8DD97CC74180EE26%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35&CFID=784797565&CFTOKEN=41036537&__acm__=1499865974_286433830c9da409d79f293351437853', 'http://delivery.acm.org/10.1145/2100000/2090122/p44-verbert.pdf?ip=14.139.187.130&id=2090122&acc=ACTIVE%20SERVICE&key=045416EF4DDA69D9%2E8DD97CC74180EE26%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35&CFID=784797565&CFTOKEN=4

In [13]:
print wordnet_similarity('https://en.wikipedia.org/wiki/Recommender_system','https://www.digitalocean.com/community/tutorials/how-to-set-up-a-host-name-with-digitalocean')


0.0553271028037
